In [1]:
import json

def deduplicate_json(input_file, output_file):
    # 读取JSON文件
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 使用字典进行去重
    unique_dict = {}
    for item in data:
        key = (item['code'], item['name'])
        if key not in unique_dict:
            unique_dict[key] = item

    # 将去重后的数据转换回列表
    deduplicated_data = list(unique_dict.values())

    # 写入新的JSON文件
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(deduplicated_data, f, ensure_ascii=False, indent=2)

    print(f"原始数据数量: {len(data)}")
    print(f"去重后数据数量: {len(deduplicated_data)}")

# 使用函数
input_file = 'hushen300.json'
output_file = 'hushen300_deduplicated.json'
deduplicate_json(input_file, output_file)

原始数据数量: 300
去重后数据数量: 282


In [10]:
import akshare as ak
import pandas as pd

def get_hushen300_stocks():
    """
    获取沪深300成分股列表，返回包含股票代码和名称的DataFrame，
    其中股票代码列名为 'code'，股票名称列名为 'name'。
    对股票代码进行去重，保留首次出现的记录。
    """
    df = ak.index_stock_cons(symbol="000300")
    stocks = pd.DataFrame({
        'code': df['品种代码'],
        'name': df['品种名称']
    })
    # 根据code去重，保留第一次出现的记录
    stocks = stocks.drop_duplicates(subset=['code'], keep='first').reset_index(drop=True)
    return stocks

# 使用示例
hushen300_stocks = get_hushen300_stocks()
print(hushen300_stocks)
as_unique_codes = hushen300_stocks['code'].tolist()
print(type(as_unique_codes[0]))

       code  name
0    302132  中航成飞
1    001391   国货航
2    688047  龙芯中科
3    002600  领益智造
4    601077  渝农商行
..      ...   ...
277  600030  中信证券
278  600031  三一重工
279  600036  招商银行
280  600050  中国联通
281  000002   万科A

[282 rows x 2 columns]
<class 'str'>


# Diff两个json的代码

In [14]:
def compare_json(json1_path, json2_path):
    with open(json1_path, 'r', encoding='utf-8') as f1, open(json2_path, 'r', encoding='utf-8') as f2:
        data1 = json.load(f1)
        data2 = json.load(f2)

    code_data1 = {item["code"]: item for item in data1["signals"]}
    code_data2 = {item["code"]: item for item in data2["signals"]}

    all_codes = set(code_data1.keys()).union(set(code_data2.keys()))
    diffs = []
    for code in all_codes:
        item1 = code_data1.get(code, {})
        item2 = code_data2.get(code, {})
        keys1 = set(item1.keys())
        keys2 = set(item2.keys())
        all_keys = keys1.union(keys2)
        for key in all_keys:
            if key != "code" and item1.get(key) != item2.get(key):
                diffs.append({
                    "code": code,
                    "key": key,
                    "value_in_json1": item1.get(key),
                    "value_in_json2": item2.get(key)
                })
    return diffs

# 假设json1.json和json2.json是你要比较的两个文件路径
json1_path = '../log/signal-linux-qstock.json'
json2_path = '../log/signal-linux-akshare.json'
differences = compare_json(json1_path, json2_path)
for diff in differences:
    print(diff)

{'code': '600085', 'key': 'signals', 'value_in_json1': {'顶钝化': False, '底钝化': False, '顶结构': False, '底结构': False, '顶背离': False, '底背离': False, '主升': False, '顶成立': False, '底成立': False}, 'value_in_json2': None}
{'code': '600085', 'key': 'close', 'value_in_json1': 35.37, 'value_in_json2': None}
{'code': '600085', 'key': 'name', 'value_in_json1': '同仁堂', 'value_in_json2': None}
{'code': '600085', 'key': 'date', 'value_in_json1': '2025-07-16', 'value_in_json2': None}
{'code': '600027', 'key': 'signals', 'value_in_json1': {'顶钝化': False, '底钝化': False, '顶结构': False, '底结构': False, '顶背离': False, '底背离': False, '主升': False, '顶成立': False, '底成立': False}, 'value_in_json2': {'顶钝化': False, '底钝化': True, '顶结构': False, '底结构': False, '顶背离': False, '底背离': True, '主升': False, '顶成立': False, '底成立': False}}
{'code': '600690', 'key': 'name', 'value_in_json1': '海尔智家', 'value_in_json2': '青岛海尔'}
{'code': '600406', 'key': 'signals', 'value_in_json1': {'顶钝化': False, '底钝化': False, '顶结构': False, '底结构': False, '顶背离': False, 

# QS接口的沪深300

In [11]:
import json

# 读取并提取股票代码（最简版本）
with open('../log/signal-linux-qstock.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 提取所有code并去重排序
qs_unique_codes = sorted(list(set(item['code'] for item in data['signals'])))

# 打印结果
print(f"共提取到 {len(qs_unique_codes)} 个唯一股票代码:")
for i, code in enumerate(qs_unique_codes, 1):
    print(f"{code:8}", end="")
    if i % 10 == 0:  # 每10个换行
        print()

print(f"\n\n代码列表: {qs_unique_codes}")
print(type(qs_unique_codes[0]))

共提取到 300 个唯一股票代码:
000001  000002  000063  000100  000157  000166  000301  000333  000338  000408  
000425  000538  000568  000596  000617  000625  000630  000651  000661  000708  
000725  000768  000776  000786  000792  000800  000807  000858  000876  000895  
000938  000963  000975  000977  000983  000999  001289  001391  001965  001979  
002001  002027  002028  002049  002050  002074  002129  002142  002179  002180  
002230  002236  002241  002252  002304  002311  002352  002371  002415  002422  
002459  002460  002463  002466  002475  002493  002594  002600  002601  002648  
002709  002714  002736  002916  002920  002938  003816  300014  300015  300033  
300059  300122  300124  300274  300308  300316  300347  300394  300408  300413  
300418  300433  300442  300498  300502  300628  300661  300750  300759  300760  
300782  300832  300896  300979  300999  301236  301269  302132  600000  600009  
600010  600011  600015  600016  600018  600019  600023  600025  600026  600027  
600028  60

In [13]:
# 检查 as_unique_codes 中每个元素是否都出现在 qs_unique_codes 中
all_in = all(code in qs_unique_codes for code in as_unique_codes)
print(f"as_unique_codes 中的所有元素是否都在 qs_unique_codes 中: {all_in}")

as_unique_codes 中的所有元素是否都在 qs_unique_codes 中: True


In [4]:
import pandas as pd

# 创建示例DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 5],
    'B': ['a', 'b', 'c', 'd', 'e'],
    'C': [10.1, 20.2, 30.3, 40.4, 50.5]
})

print("原始DataFrame:")
print(df)
print()

# 访问第2行第1列的元素（索引从0开始）
element = df.iloc[1, 0]  # 第2行，第1列
print(f"第2行第1列的元素: {element}")  # 输出: 2

# 访问第3行第2列的元素
element = df.iloc[2, 1]  # 第3行，第2列
print(f"第3行第2列的元素: {element}")  # 输出: c

原始DataFrame:
   A  B     C
0  1  a  10.1
1  2  b  20.2
2  3  c  30.3
3  4  d  40.4
4  5  e  50.5

第2行第1列的元素: 2
第3行第2列的元素: c


In [3]:
# 假设我们有以下股票数据
stock_data = pd.DataFrame({
    '日期': ['2025-07-15', '2025-07-16', '2025-07-17', '2025-07-18', '2025-07-19'],
    '开盘': [10.0, 10.2, 10.1, 10.3, 10.5],
    '最高': [10.5, 10.8, 10.6, 10.9, 11.0],
    '最低': [9.8, 10.0, 9.9, 10.1, 10.3],
    '收盘': [10.2, 10.6, 10.4, 10.7, 10.9],
    '成交量': [1000, 1200, 1100, 1300, 1400]
})

print("股票数据:")
print(stock_data)
print()

# 获取最新一天的数据（最后一行）
latest_data = stock_data.iloc[-1]
print("最新一天数据:")
print(latest_data)
print()

# 获取最近3天的收盘价
recent_closes = stock_data.iloc[-3:, 4]  # 最后3行，第5列（收盘价）
print("最近3天收盘价:")
print(recent_closes)
print()

# 获取第2-4天的所有数据
middle_data = stock_data.iloc[1:4, :]
print("第2-4天数据:")
print(middle_data)

股票数据:
           日期    开盘    最高    最低    收盘   成交量
0  2025-07-15  10.0  10.5   9.8  10.2  1000
1  2025-07-16  10.2  10.8  10.0  10.6  1200
2  2025-07-17  10.1  10.6   9.9  10.4  1100
3  2025-07-18  10.3  10.9  10.1  10.7  1300
4  2025-07-19  10.5  11.0  10.3  10.9  1400

最新一天数据:
日期     2025-07-19
开盘           10.5
最高           11.0
最低           10.3
收盘           10.9
成交量          1400
Name: 4, dtype: object

最近3天收盘价:
2    10.4
3    10.7
4    10.9
Name: 收盘, dtype: float64

第2-4天数据:
           日期    开盘    最高    最低    收盘   成交量
1  2025-07-16  10.2  10.8  10.0  10.6  1200
2  2025-07-17  10.1  10.6   9.9  10.4  1100
3  2025-07-18  10.3  10.9  10.1  10.7  1300


In [9]:
def calculate_increase(now, old):
    if old == 0:
        return None
    increase = (now - old) / old * 100
    return f"{increase:.2f}%"

print(calculate_increase(5.45, 5.38))

1.30%
